### CNN-based 3D Skeleton Tennis Shot Recognition

Based on method from paper ['Skeleton-based Action Recognition with Convolutional Neural Networks'](https://arxiv.org/abs/1704.07595) by Li et al. (2017).

We use the 3D skeletal animation data from the ['THETIS'](http://thetis.image.ece.ntua.gr/) dataset for training. Each animation is represented as a `T x N` 3-channel image where `T` is the number of frames of the animation, `N` is the number of skeletal joints, and the 3 colour channels represent the `x, y, z` coordinates of the joints.

Load the skeleton images training data.

In [1]:
import tensorflow as tf

skeleton_dir = "data/skeleton_images"

img_height = 32
img_width = 32
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
  skeleton_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  skeleton_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 1974 files belonging to 4 classes.
Using 1580 files for training.
Found 1974 files belonging to 4 classes.
Using 394 files for validation.
['backhand', 'forehand', 'service', 'smash']


Create the CNN model.

In [2]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

model = Sequential([
  layers.Rescaling(1./127.5, offset=-1, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

Compile and train the model.

In [3]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

model.save('cnn_recognition_model.h5')

Epoch 1/10
50/50 [==============================] - 2s 27ms/step - loss: 0.9145 - accuracy: 0.6595 - val_loss: 0.6230 - val_accuracy: 0.7665
Epoch 2/10
50/50 [==============================] - 1s 16ms/step - loss: 0.5052 - accuracy: 0.8057 - val_loss: 0.4474 - val_accuracy: 0.8198
Epoch 3/10
50/50 [==============================] - 1s 16ms/step - loss: 0.4094 - accuracy: 0.8367 - val_loss: 0.4192 - val_accuracy: 0.8350
Epoch 4/10
50/50 [==============================] - 1s 16ms/step - loss: 0.3320 - accuracy: 0.8608 - val_loss: 0.3245 - val_accuracy: 0.8807
Epoch 5/10
50/50 [==============================] - 1s 15ms/step - loss: 0.2717 - accuracy: 0.8899 - val_loss: 0.2849 - val_accuracy: 0.8985
Epoch 6/10
50/50 [==============================] - 1s 15ms/step - loss: 0.2173 - accuracy: 0.9127 - val_loss: 0.2391 - val_accuracy: 0.9061
Epoch 7/10
50/50 [==============================] - 1s 15ms/step - loss: 0.2089 - accuracy: 0.9146 - val_loss: 0.2562 - val_accuracy: 0.9036
Epoch 8/10
50